Test of library Requests with boursorama 

In [2]:
import requests
import pandas as pd
url = f"https://www.boursorama.com/bourse/trackers/cours/historique/1rTWPEA"

response = requests.get(url, timeout=5)  # 5 secondes max
df_list = pd.read_html(response.text)
pd.DataFrame(df_list[0])

,Date,Dernier,Var. %,+ haut,+ bas,Ouverture
0,27/05/2025,547,"+0,76%",548,543,544
1,26/05/2025,543,"+0,90%",544,533,543
2,23/05/2025,538,"-1,03%",544,533,543
3,22/05/2025,544,"-0,75%",545,540,543
4,21/05/2025,548,"-0,83%",549,544,548
5,20/05/2025,553,"+0,23%",554,551,552
6,19/05/2025,551,"-0,56%",552,545,549
7,16/05/2025,554,"+0,75%",554,551,551
8,15/05/2025,550,"+0,38%",551,545,546
9,14/05/2025,548,"-0,31%",550,545,549


il faut voir un moyen de passer de l'ISIN au ticker
Par défaut, on a un mois d'histo. Pour avoir plus, il faut:
- choisir 3Y 
- appuyer sur rechercher 
- appuyer sur le numero de la page + 1

Request nous permet de lire l'html mais pas faire du dynamique. Pour simuler du click bouton, il faut utiliser Selenium. 


In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time

url="https://www.boursorama.com/bourse/trackers/cours/historique/1rTWPEA"
driver = webdriver.Chrome()

# Ouvre la page
driver.get(url) 
# Pause pour observer 
time.sleep(2)
driver.quit()

Problème identifié: pop up des cookies à gérer

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
import time

url="https://www.boursorama.com/bourse/trackers/cours/historique/1rTWPEA"
driver = webdriver.Chrome()

# Ouvre la page
driver.get(url)  # Remplace par ton URL
wait = WebDriverWait(driver, 10)

try:
    continue_button = wait.until(EC.element_to_be_clickable(
        (By.CLASS_NAME, "didomi-continue-without-agreeing")
    ))
    continue_button.click()
    print("Bouton 'Continuer sans accepter' cliqué.")
except Exception as e:
    print("Bouton cookies pas trouvé ou déjà accepté :", e)

time.sleep(5)

driver.quit()

Bouton 'Continuer sans accepter' cliqué.


In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
import time

url="https://www.boursorama.com/bourse/trackers/cours/historique/1rTWPEA"
driver = webdriver.Chrome()

# Ouvre la page
driver.get(url)  # Remplace par ton URL
wait = WebDriverWait(driver, 10)

try:
    continue_button = wait.until(EC.element_to_be_clickable(
        (By.CLASS_NAME, "didomi-continue-without-agreeing")
    ))
    continue_button.click()
    print("Bouton 'Continuer sans accepter' cliqué.")
except Exception as e:
    print("Bouton cookies pas trouvé ou déjà accepté :", e)

time.sleep(5)

# Sélectionner "3 ans"
select_element = wait.until(EC.visibility_of_element_located((By.ID, "historic_search_duration")))
select = Select(select_element)
select.select_by_value("3Y")

time.sleep(5)

driver.quit()

Bouton 'Continuer sans accepter' cliqué.


TimeoutException: Message: 


Je sens que cette piste va me prendre beaucoup de temps, donc je vais chercher une API.
Recours à l'API https://marketstack.com/dashboard 

In [3]:
from dotenv import load_dotenv
import os
load_dotenv()
API_KEY=os.getenv("API_key_market_stack")

In [6]:
import requests

url = f"https://api.marketstack.com/v1/eod?access_key={API_KEY}"

querystring = {"symbols":"AAPL","date_from":"2025-05-20" ,"date_to":"2025-05-27"}

response = requests.get(url, params=querystring)

print(response.json())

{'pagination': {'limit': 100, 'offset': 0, 'count': 5, 'total': 5}, 'data': [{'open': 198.3, 'high': 200.74, 'low': 197.43, 'close': 200.21, 'volume': 56229000.0, 'adj_high': 200.74, 'adj_low': 197.43, 'adj_close': 200.21, 'adj_open': 198.3, 'adj_volume': 56288475.0, 'split_factor': 1.0, 'dividend': 0.0, 'symbol': 'AAPL', 'exchange': 'XNAS', 'date': '2025-05-27T00:00:00+0000'}, {'open': 193.665, 'high': 197.7, 'low': 193.46, 'close': 195.27, 'volume': 78432918.0, 'adj_high': 197.7, 'adj_low': 193.46, 'adj_close': 195.27, 'adj_open': 193.665, 'adj_volume': 78432918.0, 'split_factor': 1.0, 'dividend': 0.0, 'symbol': 'AAPL', 'exchange': 'XNAS', 'date': '2025-05-23T00:00:00+0000'}, {'open': 200.71, 'high': 202.75, 'low': 199.7, 'close': 201.36, 'volume': 46633900.0, 'adj_high': 202.75, 'adj_low': 199.7, 'adj_close': 201.36, 'adj_open': 200.71, 'adj_volume': 46742407.0, 'split_factor': 1.0, 'dividend': 0.0, 'symbol': 'AAPL', 'exchange': 'XNAS', 'date': '2025-05-22T00:00:00+0000'}, {'open': 

In [8]:
json_response=response.json()

In [12]:
import pandas as pd
pd.DataFrame(json_response['data'])

,open,high,low,close,volume,adj_high,adj_low,adj_close,adj_open,adj_volume,split_factor,dividend,symbol,exchange,date
0,198.300,200.74,197.43,200.21,56229000.0,200.74,197.43,200.21,198.300,56288475.0,1.0,0.0,AAPL,XNAS,2025-05-27T00:00:00+0000
1,193.665,197.70,193.46,195.27,78432918.0,197.70,193.46,195.27,193.665,78432918.0,1.0,0.0,AAPL,XNAS,2025-05-23T00:00:00+0000
2,200.710,202.75,199.70,201.36,46633900.0,202.75,199.70,201.36,200.710,46742407.0,1.0,0.0,AAPL,XNAS,2025-05-22T00:00:00+0000
3,205.170,207.04,200.71,202.09,59134800.0,207.04,200.71,202.09,205.170,59211774.0,1.0,0.0,AAPL,XNAS,2025-05-21T00:00:00+0000
4,207.670,208.47,205.03,206.86,42443700.0,208.47,205.03,206.86,207.670,42496635.0,1.0,0.0,AAPL,XNAS,2025-05-20T00:00:00+0000


In [16]:
!pip install datautil

    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\Oumaima\anaconda3\python.exe' -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Oumaima\\AppData\\Local\\Temp\\pip-install-kfztjzld\\datautil_413361349a924dbc98b8c276f9e9a2a3\\setup.py'"'"'; __file__='"'"'C:\\Users\\Oumaima\\AppData\\Local\\Temp\\pip-install-kfztjzld\\datautil_413361349a924dbc98b8c276f9e9a2a3\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\Oumaima\AppData\Local\Temp\pip-pip-egg-info-m11kuczg'
         cwd: C:\Users\Oumaima\AppData\Local\Temp\pip-install-kfztjzld\datautil_413361349a924dbc98b8c276f9e9a2a3\
    Complete output (7 lines):
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "C:\Users\Oumaima\AppData\Local\Temp\pip-install-kfztjzld\datautil_413361349a924dbc98b8c276f9e9a2a

In [45]:
import requests
import pandas as pd

def get_hist_data(start_date_str: str, end_date_str: str, API_KEY: str, symbol : str) -> pd.DataFrame:
    
    """
    Fetches historical end-of-day stock data from the MarketStack API.

    Parameters:
    - start_date_str: str, the start date in 'YYYY-MM-DD' format.
    - end_date_str: str, the end date in 'YYYY-MM-DD' format.
    - API_KEY: str, your MarketStack API key.
    - symbol: str, the symbol of the instrument (https://marketstack.com/search to find the exact ticker)

    Returns:
    - pd.DataFrame: DataFrame containing the historical data.
    """
    
    url = f"https://api.marketstack.com/v1/eod?access_key={API_KEY}"
    # API parameters (max 1000 rows allowed per request)
    querystring = {

        "symbols": symbol,
        "date_from": end_date_str,
        "date_to": start_date_str,
        "limit": 1000
    }

    try:
        # Send GET request to the API
        print(f"Sending request to {url} with params: {querystring}")
        response = requests.get(url, params=querystring)
        response.raise_for_status()  # Raises an error for HTTP codes 4xx or 5xx

        json_response = response.json()

        # Ensure 'data' key exists in response
        if 'data' not in json_response:
            raise ValueError("Missing 'data' key in API response")

        # Convert to DataFrame
        return pd.DataFrame(json_response['data'])

    except requests.RequestException as e:
        print(f"HTTP request failed: {e}")
    except ValueError as ve:
        print(f"Value error: {ve}")
    except Exception as ex:
        print(f"Unexpected error: {ex}")

    # Return empty DataFrame in case of failure
    return pd.DataFrame()



In [46]:
from datetime import date
from dateutil.relativedelta import relativedelta
start=date.today()
three_years_ago=start - relativedelta(years=3)
start_date_str=start.strftime("%Y-%m-%d")
end_date_str=three_years_ago.strftime("%Y-%m-%d")
 df=get_hist_data(start_date_str=start_date_str,end_date_str=end_date_str,API_KEY=API_KEY,symbol="WPEA.XPAR")

In [ ]:
from dateutil.relativedelta import relativedelta


dans https://www.justetf.com/en/etf-profile.html?isin=IE0002XZSHO1

ISIN IE0002XZSHO1   |  WKN A3E1JV   |  Ticker WPEA

https://marketstack.com/search to find the exact ticker
boursorama 1rTWPEA

Le code ISIN IE0002XZSHO1 correspond à l’ETF iShares MSCI World Swap PEA UCITS ETF EUR (Acc), dont le ticker est WPEA. Cet ETF est coté sur Euronext Paris .

Nom complet : iShares MSCI World Swap PEA UCITS ETF EUR (Acc)

Ticker : WPEA

ISIN : IE0002XZSHO1

WKN : A3E1JV

Cet ETF vise à répliquer la performance de l’indice MSCI World Net Total Return EUR, qui couvre les actions de grande et moyenne capitalisation des marchés développés. Il est conçu pour être éligible au Plan d’Épargne en Actions (PEA), ce qui en fait une option intéressante pour les investisseurs français souhaitant une exposition mondiale tout en bénéficiant des avantages fiscaux du PEA.
BlackRock

In [55]:
df.shape

(256, 15)

In [27]:
start_date_str,end_date_str

('2025-05-28', '2022-05-28')

In [56]:
pd.to_datetime(df.date).describe()

count                                 256
mean     2024-11-24 04:46:52.500000+00:00
min             2024-05-28 00:00:00+00:00
25%             2024-08-25 06:00:00+00:00
50%             2024-11-21 12:00:00+00:00
75%             2025-02-24 06:00:00+00:00
max             2025-05-28 00:00:00+00:00
Name: date, dtype: object

Que un an d'histo ....

In [58]:
df_copy=df.copy()

In [59]:
df.date=pd.to_datetime(df.date)

In [61]:
df['close'].std()

0.3018661293819078

In [62]:
df['close'].rolling(5).std()

0           NaN
1           NaN
2           NaN
3           NaN
4      0.038552
         ...   
251    0.044736
252    0.042296
253    0.030442
254    0.016603
255    0.023141
Name: close, Length: 256, dtype: float64

In [83]:
one_years_ago=start - relativedelta(years=1)
cond_inf=(df['date'].dt.date>one_years_ago)
cond_sup=(df['date'].dt.date<date.today())

df.loc[cond_sup & cond_inf,'close'].std()

0.3015711336248684

In [104]:
returns = sub_df['close'].pct_change()
log_returns = np.log(returns / returns.shift(1))
log_returns.shape[0]

C:\Users\Oumaima\anaconda3\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


(255,)

In [105]:
log_returns = np.log(sub_df['close'] / sub_df['close'].shift(1))
mean_daily_return = log_returns.mean()
expected_annual_return = mean_daily_return * log_returns.shape[0]

In [106]:
expected_annual_return, mean_daily_return

(-0.08527937620019578, -0.00033442892627527757)

In [110]:
import numpy as np
def get_stats(indicator, min_date, max_date):
    cond_inf=(df['date'].dt.date>min_date)
    cond_sup=(df['date'].dt.date<max_date)
    sub_df=df.loc[cond_inf & cond_sup,:]
    if indicator=='vol':
        return sub_df.loc[:,'close'].std()
    elif indicator=='performance':
        return (sub_df['close'].iloc[-1] / sub_df['close'].iloc[0]) - 1
    elif indicator=='rendement espéré':
        log_returns = np.log(sub_df['close'] / sub_df['close'].shift(1))
        mean_daily_return = log_returns.mean()
        expected_annual_return = mean_daily_return * log_returns.shape[0]

        return expected_annual_return
    elif indicator=='max dropdown':
        # Série du pic cumulé jusqu'à présent
        running_max = df['close'].cummax()
        # Drawdown actuel
        drawdown = (df['close'] - running_max) / running_max
        # Max drawdown (le plus bas drawdown, valeur négative)
        max_drawdown = drawdown.min()
        return max_drawdown
    else:
        print('Unknown indicator')
        return None
        

In [115]:
import numpy as np
import pandas as pd
from datetime import date

def get_stats(indicator: str, min_date: date, max_date: date) -> float:
    """
    Calcule différents indicateurs financiers sur une période donnée à partir du DataFrame global `df`.

    Paramètres :
    - indicator : str, le nom de l'indicateur à calculer ('vol', 'performance', 'rendement espéré', 'max dropdown')
    - min_date : date de début de la période (exclue)
    - max_date : date de fin de la période (exclue)

    Retour :
    - float : la valeur de l'indicateur, ou None si l'indicateur est inconnu
    """
    
    # Filtrer les données entre min_date et max_date
    cond_inf = (df['date'].dt.date > min_date)
    cond_sup = (df['date'].dt.date < max_date)
    sub_df = df.loc[cond_inf & cond_sup, :]

    if indicator == 'vol':
        # Volatilité = écart-type des prix de clôture
        return sub_df['close'].std()

    elif indicator == 'performance':
        # Performance = variation relative entre le 1er et le dernier prix
        return (sub_df['close'].iloc[-1] / sub_df['close'].iloc[0]) - 1

    elif indicator == 'rendement espéré':
        # Calcul des rendements log
        log_returns = np.log(sub_df['close'] / sub_df['close'].shift(1))
        # Moyenne des rendements journaliers (supposés réguliers ici)
        mean_daily_return = log_returns.mean()
        # Annualisation : ici on multiplie par le nombre d'observations (≈ nombre de jours)
        expected_annual_return = mean_daily_return * log_returns.shape[0]
        return expected_annual_return

    elif indicator == 'max dropdown':
        # Série du maximum cumulé des prix
        running_max = sub_df['close'].cummax()
        # Drawdown = écart relatif entre le prix courant et le max précédent
        drawdown = (sub_df['close'] - running_max) / running_max
        # Max drawdown = drawdown le plus bas (plus forte perte)
        max_drawdown = drawdown.min()
        return max_drawdown

    else:
        print('Unknown indicator')
        return None


In [120]:
from datetime import date
from dateutil.relativedelta import relativedelta
import pandas as pd

# Horizons à évaluer
periods = {
    'YTD': date(date.today().year, 1, 1),
    '3M': date.today() - relativedelta(months=3),
    '6M': date.today() - relativedelta(months=6),
    '1Y': date.today() - relativedelta(years=1),
    '3Y': date.today() - relativedelta(years=3),
}

# Liste des indicateurs
indicators = ['vol', 'performance', 'rendement espéré', 'max dropdown']

# Initialiser une liste pour stocker les résultats
results = []

# Boucle sur chaque période et chaque indicateur
for label, min_date in periods.items():
    for indicator in indicators:
        value = get_stats(indicator, min_date, date.today())
        results.append({
            'Période': label,
            'Indicateur': indicator,
            'Valeur': value
        })

# Convertir les résultats en DataFrame
df_results = pd.DataFrame(results)

# Option : pivot pour avoir un tableau lisible
df_pivot = df_results.pivot(index='Indicateur', columns='Période', values='Valeur')

# Affichage
print(df_pivot)

# Export CSV (facultatif)
# df_pivot.to_csv("indicateurs_performance.csv", float_format="%.6f")


Période                 1Y        3M        3Y        6M       YTD
Indicateur                                                        
max dropdown     -0.184254 -0.154523 -0.184254 -0.154523 -0.154523
performance      -0.080380  0.060614 -0.076982  0.052302  0.056321
rendement espéré -0.084126  0.059862 -0.080422  0.051401  0.055340
vol               0.302145  0.249430  0.302437  0.328470  0.344292


In [121]:
df_pivot

Période,1Y,3M,3Y,6M,YTD
Indicateur,,,,,
max dropdown,-0.184254,-0.154523,-0.184254,-0.154523,-0.154523
performance,-0.080380,0.060614,-0.076982,0.052302,0.056321
rendement espéré,-0.084126,0.059862,-0.080422,0.051401,0.055340
vol,0.302145,0.249430,0.302437,0.328470,0.344292


In [116]:
one_years_ago=start - relativedelta(years=1)
cond_inf=(df['date'].dt.date>one_years_ago)
cond_sup=(df['date'].dt.date<date.today())
get_stats('vol',one_years_ago,date.today())

0.3021446469552309

In [117]:
get_stats('max dropdown',one_years_ago,date.today())

-0.18425410206232115

In [118]:
get_stats('rendement espéré',one_years_ago,date.today())

-0.08412591830815108

In [119]:
get_stats('performance',one_years_ago,date.today())

-0.08037997807818786

In [ ]:
import numpy as np

# Rendements simples quotidiens
df['return'] = df['price'].pct_change()

# Ou rendements log (plus courants en finance)
df['log_return'] = np.log(df['price'] / df['price'].shift(1))
